# Assignment 4: Exploratory Data Analysis 3: Combined Preprocessing

### Read in Datasets: Training, Address, Latlons

In [1]:
# reading in data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

# reading in training data
blight_train = pd.read_csv('readonly/train.csv', encoding='cp1252')
addresses = pd.read_csv('readonly/addresses.csv')
latlons = pd.read_csv('readonly/latlons.csv')

# checking training data
blight_train.head()

C:\Users\drewn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


#### Combining Datasets

In [2]:
# merge 1: setting indices to ticket id
blight_train = blight_train.set_index('ticket_id')
addresses = addresses.set_index('ticket_id')

# merge 1: merging on ticket id
blight_train = pd.merge(blight_train, addresses, how = 'inner',
                    left_index=True, right_index=True)

# resetting index, to retain ticket id in data
blight_train = blight_train.reset_index()

# merge 2: setting address to upper case
blight_train['address'] = blight_train['address'].str.upper()
latlons['address'] = latlons['address'].str.upper()

# merge 2: setting indices to address
blight_train = blight_train.set_index('address')
latlons = latlons.set_index('address')

# merge 2: merging on address
blight_train = pd.merge(blight_train, latlons, how = 'inner',
                    left_index=True, right_index=True)

# resetting index
blight_train = blight_train.reset_index()
blight_train.head()

,address,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
0,"0 10TH ST, DETROIT MI",24928,Department of Public Works,"Hayes, Billy J","XXX, XXX XXX",0.0,10TH ST,NaN,NaN,XXXX,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.325689,-83.06433
1,"0 10TH ST, DETROIT MI",71887,Health Department,"Zizi, Josue","0000000000000000000000, 000000000000000000000",0.0,10TH ST,NaN,NaN,0,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.325689,-83.06433
2,"0 10TH ST, DETROIT MI",157969,Department of Public Works,"Baker, Kenneth","VOID, VOID",0.0,10TH ST,NaN,NaN,VOID,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.325689,-83.06433
3,"0 10TH ST, DETROIT MI",168151,Department of Public Works,"Zizi, Josue","0000000000000, 00000000000000",0.0,10TH ST,NaN,NaN,0,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.325689,-83.06433
4,"0 10TH ST, DETROIT MI",195261,Department of Public Works,"Zizi, Josue","VOID, VOID",0.0,10TH ST,NaN,NaN,VOID,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.325689,-83.06433


In [3]:
# Dropping all variables only in training set
no_feat = ['balance_due', 'collection_status', 'compliance_detail', 'payment_amount', 'payment_date', 'payment_status']
blight_train.drop(no_feat, axis=1, inplace=True)
blight_train.head()

,address,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,...,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance,lat,lon
0,"0 10TH ST, DETROIT MI",24928,Department of Public Works,"Hayes, Billy J","XXX, XXX XXX",0.0,10TH ST,NaN,NaN,XXXX,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,42.325689,-83.06433
1,"0 10TH ST, DETROIT MI",71887,Health Department,"Zizi, Josue","0000000000000000000000, 000000000000000000000",0.0,10TH ST,NaN,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,42.325689,-83.06433
2,"0 10TH ST, DETROIT MI",157969,Department of Public Works,"Baker, Kenneth","VOID, VOID",0.0,10TH ST,NaN,NaN,VOID,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,42.325689,-83.06433
3,"0 10TH ST, DETROIT MI",168151,Department of Public Works,"Zizi, Josue","0000000000000, 00000000000000",0.0,10TH ST,NaN,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,42.325689,-83.06433
4,"0 10TH ST, DETROIT MI",195261,Department of Public Works,"Zizi, Josue","VOID, VOID",0.0,10TH ST,NaN,NaN,VOID,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,42.325689,-83.06433


In [4]:
# Dropping all unused categorical features
cat_feat = ['address', 'city', 'disposition', 'grafitti_status', 'hearing_date', 'mailing_address_str_name', 'non_us_str_code', 'ticket_issued_date', 'violation_code', 'violation_street_name', 'violator_name', 'zip_code']
blight_train.drop(cat_feat, axis=1, inplace=True)
blight_train.head()

,ticket_id,agency_name,inspector_name,violation_street_number,violation_zip_code,mailing_address_str_number,state,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon
0,24928,Department of Public Works,"Hayes, Billy J",0.0,NaN,NaN,MI,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
1,71887,Health Department,"Zizi, Josue",0.0,NaN,NaN,MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
2,157969,Department of Public Works,"Baker, Kenneth",0.0,NaN,NaN,MI,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
3,168151,Department of Public Works,"Zizi, Josue",0.0,NaN,NaN,MI,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
4,195261,Department of Public Works,"Zizi, Josue",0.0,NaN,NaN,MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433


In [5]:
# Dropping all unused continuous features
cont_feat = ['clean_up_cost', 'mailing_address_str_number', 'ticket_id', 'violation_street_number', 'violation_zip_code']
blight_train.drop(cont_feat, axis=1, inplace=True)
blight_train.head()

,agency_name,inspector_name,state,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon
0,Department of Public Works,"Hayes, Billy J",MI,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
1,Health Department,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
2,Department of Public Works,"Baker, Kenneth",MI,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
3,Department of Public Works,"Zizi, Josue",MI,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433
4,Department of Public Works,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,NaN,42.325689,-83.06433


### Exploring Continuous Features

In [6]:
blight_train.describe().T

,count,mean,std,min,25%,50%,75%,max
fine_amount,250305.0,374.423435,707.195807,0.000000,200.000000,250.000000,250.000000,10000.000000
admin_fee,250306.0,12.774764,9.607344,0.000000,0.000000,20.000000,20.000000,20.000000
state_fee,250306.0,6.387382,4.803672,0.000000,0.000000,10.000000,10.000000,10.000000
late_fee,250306.0,21.494506,56.464263,0.000000,0.000000,10.000000,25.000000,1000.000000
discount_amount,250306.0,0.125167,3.430178,0.000000,0.000000,0.000000,0.000000,350.000000
judgment_amount,250306.0,268.685356,626.915212,0.000000,0.000000,140.000000,305.000000,11030.000000
compliance,159880.0,0.072536,0.259374,0.000000,0.000000,0.000000,0.000000,1.000000
lat,250303.0,42.389139,0.038240,41.785926,42.364044,42.392079,42.418848,45.809387
lon,250303.0,-83.111452,0.094370,-88.081348,-83.185490,-83.123886,-83.035374,-82.433593


In [7]:
# summarizing key features by compliance
blight_train.groupby('compliance').mean()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,lat,lon
compliance,,,,,,,,
0.0,366.319281,20.0,10.0,35.261677,0.008700,431.580958,42.389970,-83.111518
1.0,238.327240,20.0,10.0,13.063430,2.590325,280.886229,42.386884,-83.111674


##### Notes on above groupby table

There are some very interesting things to note here

* Compliance is associated with:
 * lower fine amounts
 * lower late fees
 * higher discount amounts
 * lower judgement amounts

* It is likely that the direction of correlation is as follows:

 * Variable => Compliance: lower fine amounts, higher discount amounts, lower judgement amounts
 * Compliance => Variable: lower late fees

In [8]:
# checking variable averages for null values of compliance
blight_train.groupby(blight_train['compliance'].isnull()).mean()

# not conducting any imputation or histogram plots at this point

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon
compliance,,,,,,,,,
False,357.035295,20.0,10.0,33.651512,0.195959,420.650218,0.072536,42.389747,-83.111530
True,405.167321,0.0,0.0,0.000000,0.000000,0.000000,NaN,42.388065,-83.111316


In [9]:
# checking overall compliance rate
blight_train['compliance'].mean()

# compliance rate is roughly 7.25%

0.07253565173880411

### Exploring Categorical Features

Exploring `agency_name`, `city`, `state`, `country`

In [10]:
blight_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   agency_name            250306 non-null  object 
 1   inspector_name         250306 non-null  object 
 2   state                  250213 non-null  object 
 3   country                250306 non-null  object 
 4   violation_description  250306 non-null  object 
 5   fine_amount            250305 non-null  float64
 6   admin_fee              250306 non-null  float64
 7   state_fee              250306 non-null  float64
 8   late_fee               250306 non-null  float64
 9   discount_amount        250306 non-null  float64
 10  judgment_amount        250306 non-null  float64
 11  compliance             159880 non-null  float64
 12  lat                    250303 non-null  float64
 13  lon                    250303 non-null  float64
dtypes: float64(9), object(5)
memory usag

In [11]:
blight_train.isnull().sum()

agency_name                  0
inspector_name               0
state                       93
country                      0
violation_description        0
fine_amount                  1
admin_fee                    0
state_fee                    0
late_fee                     0
discount_amount              0
judgment_amount              0
compliance               90426
lat                          3
lon                          3
dtype: int64

In [12]:
# checking null values for state
blight_train.groupby(blight_train['state'].isnull()).mean()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon
state,,,,,,,,,
False,374.452296,12.772798,6.386399,21.492663,0.125213,268.665724,0.072524,42.389139,-83.111468
True,296.774194,18.064516,9.032258,26.451613,0.000000,321.505376,0.095238,42.390710,-83.069404


In [13]:
# replacing null values for state with 'MI', after verifying using address 
blight_train['state'] = np.where(blight_train['state'].isnull(), 'MI', blight_train['state'])

In [14]:
# replacing remaining nan's with 0 in dataframe
blight_train = blight_train.fillna(0)

In [15]:
# checking mean after replacing nulls
blight_train.compliance.mean() 
# ok so it turns out compliance is wayy lower than i'd thought earlier
# which makes sense judging by the data
# it is not 7.25% but is in fact 4.63%

0.046331290500427474

##### Pivot Tables

In [16]:
# pivot table of compliance by agency and compliance status
blight_train.pivot_table('compliance', index='state', columns='agency_name', aggfunc='count')

agency_name,"Buildings, Safety Engineering & Env Department",Department of Public Works,Detroit Police Department,Health Department,Neighborhood City Halls
state,,,,,
AK,44.0,33.0,1.0,1.0,NaN
AL,135.0,53.0,18.0,9.0,NaN
AR,30.0,51.0,1.0,NaN,NaN
AZ,161.0,143.0,10.0,15.0,NaN
BC,4.0,13.0,NaN,NaN,NaN
BL,NaN,2.0,NaN,NaN,NaN
CA,1771.0,2918.0,70.0,261.0,NaN
CO,65.0,80.0,NaN,2.0,NaN
CT,18.0,42.0,NaN,NaN,NaN


### String Extraction: Violation Description

In [17]:
## Using string extraction (first 3 words of description) to reduce number of categories present in violation description

# for now, just encoding all 250 categories present in the violation description feature
blight_train.head()

,agency_name,inspector_name,state,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon
0,Department of Public Works,"Hayes, Billy J",MI,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433
1,Health Department,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433
2,Department of Public Works,"Baker, Kenneth",MI,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433
3,Department of Public Works,"Zizi, Josue",MI,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433
4,Department of Public Works,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433


### One-Hot Encoding `Agency Name`, `Inspector Name`, `State`, `Country`, and `Violation Description`

##### Agency Name

In [18]:
# Get one hot encoding of column Agency Name
one_hot = pd.get_dummies(blight_train['agency_name'])
# Drop column Agency Name as it is now encoded
blight_train = blight_train.drop('agency_name',axis = 1)
# Join the encoded df
blight_train = blight_train.join(one_hot)
blight_train.head()

,inspector_name,state,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon,"Buildings, Safety Engineering & Env Department",Department of Public Works,Detroit Police Department,Health Department,Neighborhood City Halls
0,"Hayes, Billy J",MI,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,1,0,0,0
1,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,0,0,1,0
2,"Baker, Kenneth",MI,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,1,0,0,0
3,"Zizi, Josue",MI,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,1,0,0,0
4,"Zizi, Josue",MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,1,0,0,0


##### Inspector Name

In [19]:
# Get one hot encoding of column Inspector Name
one_hot = pd.get_dummies(blight_train['inspector_name'])
# Drop column Inspector Name as it is now encoded
blight_train = blight_train.drop('inspector_name',axis = 1)
# Join the encoded df
blight_train = blight_train.join(one_hot)
blight_train.head()

,state,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,...,"Wilcox, Valerie","Wilkins, Rolland W","Williams, Cecila","Williams, Darrin","Williams, Dennis E","Williamson, Lillett","Woodall, Joseph","YOUNG, GERALDINE","Zawislak, Norbet","Zizi, Josue"
0,MI,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,MI,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,MI,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,MI,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


##### State

In [20]:
# Get one hot encoding of column State
one_hot = pd.get_dummies(blight_train['state'])
# Drop column state as it is now encoded
blight_train = blight_train.drop('state',axis = 1)
# Join the encoded df
blight_train = blight_train.join(one_hot)
blight_train.head()

,country,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,...,TX,UK,UT,VA,VI,VT,WA,WI,WV,WY
0,USA,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,...,0,0,0,0,0,0,0,0,0,0
1,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,...,0,0,0,0,0,0,0,0,0,0
2,USA,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,...,0,0,0,0,0,0,0,0,0,0
3,USA,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,...,0,0,0,0,0,0,0,0,0,0
4,USA,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,...,0,0,0,0,0,0,0,0,0,0


##### Country

In [21]:
# Get one hot encoding of column Country
one_hot = pd.get_dummies(blight_train['country'])
# Drop column country as it is now encoded
blight_train = blight_train.drop('country',axis = 1)
# Join the encoded df
blight_train = blight_train.join(one_hot)
blight_train.head()

,violation_description,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon,...,VT,WA,WI,WV,WY,Aust,Cana,Egyp,Germ,USA
0,Improper storage and separation of solid waste...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,...,0,0,0,0,0,0,0,0,0,1
1,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,...,0,0,0,0,0,0,0,0,0,1
2,Removal of snow and ice from sidewalks,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,...,0,0,0,0,0,0,0,0,0,1
3,Improperly stored or separated Solid Waste,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,...,0,0,0,0,0,0,0,0,0,1
4,"Failure of owner to keep property, its sidewal...",200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,...,0,0,0,0,0,0,0,0,0,1


##### Violation Description

In [22]:
# cleaning violation description
blight_train['violation_description2'] = ''
blight_train.loc[blight_train.violation_description2 == '', 'violation_description2'] = blight_train.violation_description.str.split().str.get(0)
blight_train.drop(['violation_description'], axis=1, inplace=True)
blight_train.head()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon,"Buildings, Safety Engineering & Env Department",...,WA,WI,WV,WY,Aust,Cana,Egyp,Germ,USA,violation_description2
0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,1,Improper
1,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,1,Failure
2,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,1,Removal
3,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,1,Improperly
4,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,1,Failure


In [23]:
# Get one hot encoding of column Violation Description
one_hot = pd.get_dummies(blight_train['violation_description2'])
# Drop column violation description 2 as it is now encoded
blight_train = blight_train.drop('violation_description2',axis = 1)
# Join the encoded df
blight_train = blight_train.join(one_hot)
blight_train.head()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,lat,lon,"Buildings, Safety Engineering & Env Department",...,Rodent,Sell,"Storage,dismantling",Transporting,Unapproved,Unlawful,Use,Using,Violation,publicly-owned
0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,0,0
1,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,0,0
2,100.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,0,0
3,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,0,0
4,200.0,0.0,0.0,0.0,0.0,0.0,0.0,42.325689,-83.06433,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# writing out cleaned dataset
blight_train.to_csv('readonly/blight_train.csv', index=False)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

features = blight_train.drop('compliance', axis=1)
labels = blight_train['compliance']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

In [ ]:
# writing out train test data for model fitting and evaluation
X_train.to_csv('model/train_features.csv', index=False)
X_test.to_csv('model/test_features.csv', index=False)

y_train.to_csv('model/train_labels.csv', index=False)
y_test.to_csv('model/test_labels.csv', index=False)